Ok so we have microegg variations

- hashmap vs indexmap
- aegraph unode but what about the linked list version of eclasses
- ac autoflattening _ built in AC matching
- extraction via top down memoization



There's a blog post to be had in "tabled memoization". On some level it's trivial because this is how you do DFS in a graph. You have to remember what you're seen or you're screwed.
DFS in dag you memoize to avoid redundancy work.
CF rational trees


In [ ]:
def extract_td(e, id):
    memo = {}
    # memo with None vs seen/open_node memo pair
    def worker(id):
        if id in memo:
            res = memo[id]
            # just return res
            if res is None:
                return None
            else:
                return res
        else:
            memo[id] = None # this branch is open
            #bestcost = None
            #for node in e.eclasses(id):
            #    args = map(worker, node.args)
            #    cost = 1 + cost(args)
            return min([App(node.f, map(worker,children))) for node in e.classes(id)], key=lambda x: cost(x))
    return worker(id)

        


In [ ]:
class ACEGraph()
    def add_ac(self, f, args):
        flatargs = [[]]
        for id in args:
            this_arg = [[id]]
            for node in self.class[id]:
                match node:
                    case AC(f1, args1) if f1 == f:
                        this_arg.append(args1)
                    case _:
                        pass
            # expand out all ways of flattening
            flatargs = [prev + this for prev in flatargs for this in this_arg]
        children = []
        for args in flatargs: # maybe dedup first. Or whatever. add_prim will do that
            seld.add_prim(AC(f, sorted(args)))

        if AC(f, sorted(args)) in memo:
            return memo
        else:
            # do eager flattening
            # add in all E equivalent flattened terms
            for sub in powerset(args)
                if AC(f, sub) in memo:
                    id = mem(AC(f, sub))
                    AC
    def rebuild():
        add in flattenings
        
    def ematch(pattern, id)::
        for node in eclasses[id]:
            match node:
                case App
                case ACApp:
                    for all partitions:
                        match

# rust


In [ ]:
use std::collections::HashMap;
use std::rc::Rc;
pub type Id = usize;
pub type Name = Rc<str>;

// The basics
#[derive(PartialEq, Eq, Hash, Clone, Debug)]
pub struct Node {
    f: Name,
    args: Vec<Id>,
}

#[derive(Default)]
pub struct EGraph {
    memo: HashMap<Node, Id>,
    uf: Vec<Id>,
}

impl EGraph {
    pub fn add_node(&mut self, node: Node) -> Id {
        let node = self.canonicalize_node(&node);
        if let Some(&id) = self.memo.get(&node) {
            id
        } else {
            let new_id = self.uf.len();
            self.uf.push(new_id);
            self.memo.insert(node, new_id);
            new_id
        }
    }

    pub fn add(&mut self, f: impl Into<Name>, args: impl Into<Vec<Id>>) -> Id {
        let node = Node {
            f: f.into(),
            args: args.into(),
        };
        self.add_node(node)
    }

    pub fn union(&mut self, a: Id, b: Id) {
        let a = self.find(a);
        let b = self.find(b);
        if a != b {
            self.uf[a] = b;
        }
    }

    pub fn find(&self, mut a: Id) -> Id {
        while self.uf[a] != a {
            a = self.uf[a];
        }
        a
    }

    pub fn is_eq(&self, a: Id, b: Id) -> bool {
        self.find(a) == self.find(b)
    }

    pub fn nodes_in_class(&self, class: Id) -> impl Iterator<Item = &Node> {
        self.memo
            .iter()
            .filter(move |(_, id)| self.is_eq(**id, class))
            .map(|(node, _)| node)
    }

    pub fn canonicalize_node(&self, node: &Node) -> Node {
        Node {
            f: node.f.clone(),
            args: node.args.iter().map(|id| self.find(*id)).collect(),
        }
    }

    pub fn rebuild(&mut self) {
        // copy needed for borrowing
        let nodes_copy = self.memo.clone();

        let mut keep_going = true;
        while keep_going {
            keep_going = false;
            for (node, id) in &nodes_copy {
                let new_node = self.canonicalize_node(node);
                let new_id = self.add_node(new_node);
                if !self.is_eq(new_id, *id) {
                    self.union(new_id, *id);
                    keep_going = true;
                }
            }
        }
        println!("rebuilt!")
    }
}

// e-matching

#[derive(PartialEq, Eq, Hash, Clone, Debug)]
pub enum Pattern {
    Var(Name),
    App(Name, Vec<Pattern>),
}

pub type Subst = HashMap<Name, Id>;

impl EGraph {
    pub fn ematch(&self, pat: &Pattern, class: Id) -> Vec<Subst> {
        self.ematch_rec(0, pat, class, Default::default())
    }
    pub fn ematch_rec(
        &self,
        depth: usize,
        pat: &Pattern,
        class: Id,
        mut subst: Subst,
    ) -> Vec<Subst> {
        println!("{:d$}subst: {subst:?}", "", d = depth * 2,);
        println!("{:d$}matching {pat:?} at {class:?}", "", d = depth * 2,);
        match pat {
            Pattern::Var(name) => {
                if let Some(&id) = subst.get(name) {
                    if self.is_eq(id, class) {
                        return vec![subst];
                    } else {
                        return vec![];
                    }
                } else {
                    subst.insert(name.clone(), class);
                    return vec![subst];
                }
            }
            Pattern::App(f, args) => {
                let mut results = vec![];
                for node in self.nodes_in_class(class) {
                    let mut todo = vec![subst.clone()];
                    println!(
                        "{:d$}matching {pat:?} at {class:?} - {node:?}",
                        " ",
                        d = depth * 2,
                    );
                    if node.f == *f && node.args.len() == args.len() {
                        for (pa, na) in args.iter().zip(node.args.iter()) {
                            todo = todo
                                .into_iter()
                                .flat_map(|subst| self.ematch_rec(depth + 1, pa, *na, subst))
                                .collect();
                        }
                    }
                    results.extend(todo);
                }
                results
            }
        }
    }
}


In [7]:

let mut eg = EGraph::default();
let a = eg.add("a", []);
let b = eg.add("b", []);
let c = eg.add("c", []);
let f1 = eg.add("f", [a, b]);
let f2 = eg.add("f", [a, c]);

eg.union(b, c);
assert!(!eg.is_eq(f1, f2));

eg.rebuild();
assert!(eg.is_eq(f1, f2));




rebuilt!


In [ ]:
fn test_match() {
    let mut eg = EGraph::default();
    let a = eg.add("a", []);
    let b = eg.add("b", []);

    let f1 = eg.add("f", [a, a]);
    let f2 = eg.add("f", [b, b]);
    let f3 = eg.add("f", [a, b]);

    eg.union(f1, f2);
    eg.union(f2, f3);
    eg.rebuild();

    use Pattern::*;
    let pat_fxx = App("f".into(), vec![Var("x".into()), Var("x".into())]);
    let pat_fxy = App("f".into(), vec![Var("x".into()), Var("y".into())]);

    let fxx_matches = eg.ematch(&pat_fxx, f1);
    assert_eq!(fxx_matches.len(), 2);

    let fxy_matches = eg.ematch(&pat_fxy, f1);
    assert_eq!(fxy_matches.len(), 3);
}